In [ ]:
# 06장 파일과 디렉터리 다루기
# -------------------------
# 프로그래밍 시 파일과 디렉터리 처리는 기본 중의 기본이다. 이번 장에서는 파일과 디렉터리를 다루는 모듈을 알아본다.

## 035 파일 경로를 객체로 다루려면? ― pathlib

In [ ]:
# 035 파일 경로를 객체로 다루려면? ― pathlib
# ----------------------------------------
# pathlib은 파일 시스템 경로를 문자열이 아닌 객체로 만들어 여러 가지 일을 할 수 있도록 하는 모듈이다.

# pathlib 모듈은 파이썬 3.4 버전부터 사용할 수 있다.

# 문제
# ----
# 다음은 현재 디렉터리의 모든 텍스트 파일(.txt)을 archive라는 디렉터리로 이동하는 일반적인 파이썬 코드이다
# (단, archive 디렉터리는 현재 디렉터리 하위에 이미 있다고 가정한다).

# import glob
# import os
# import shutil

# for file_path in glob.glob('%s/*.txt' % os.getcwd()):
#     parent = os.path.dirname(file_path)
#     filename = os.path.basename(file_path)
#     new_path = os.path.join(parent, 'archive', filename)
#     shutil.move(file_path, new_path)

# 이 코드를 glob, os, shutil 대신 pathlib만을 사용하도록 수정하려면 어떻게 해야 할까?


In [8]:
import glob
import os
import shutil

for file_path in glob.glob('%s/*.txt' % os.getcwd()):
    print(file_path)
    print('-'*70)
    
    parent = os.path.dirname(file_path)
    print(parent)
    print('-'*70)
    
    file_name = os.path.basename(file_path)
    print(file_name)
    print('-'*70)
    
    new_path = os.path.join(parent, 'archive', file_name)
    print(new_path)
    print('='*70)

    shutil.move(file_path, new_path)

In [ ]:
# 풀이
# ----
# 다음처럼 pathlib만을 사용하여 glob, os.path, shutil에서 사용했던 모든 기능을 구현할 수 있다.

# [파일명: pathlib_sample.py]

# import pathlib


# for p in pathlib.Path.cwd().glob('*.txt'):
#     new_p = p.parent.joinpath('archive', p.name)
#     p.replace(new_p)

# pathlib.Path.cwd()는 현재 디렉터리 객체(Path)를 반환한다. 
# 그리고 Path 객체의 glob() 함수로 해당 디렉터리에 있는 모든 txt 파일을 객체(Path 이터레이터)로 얻을 수 있다. 
# Path 객체의 parent()는 os.path.dirname()처럼 해당 파일의 현재 디렉터리를 반환한다. 
# 마찬가지로 Path 객체의 joinpath()는 os.path.join()처럼 파일 경로와 파일명을 합쳐 새로운 경로를 만든다. 
# 그리고 파일을 옮기고자 shutil.move() 대신 Path 객체의 replace() 함수를 사용했다.

# 하위 디렉터리까지 검색하려면 glob() 대신 rglob()을 사용하면 된다.

# pathlib를 사용하지 않은 코드와 사용한 코드를 비교하면 다음 표와 같다.

# 전통적인 방식	pathlib 방식(p는 pathlib로 생성한 Path 객체)
# os.getcwd	p.cwd
# glob.glob	p.glob
# os.path.dirname	p.parent
# os.path.basename	p.name
# os.path.join	p.joinpath
# shutil.move	p.replace


# 알아두면 좋아요
# 현재 디렉터리의 모든 파일을 조사하여 확장자별 개수 구하기
# -----------------------------------------------------
# >>> import collections, pathlib
# >>> collections.Counter([p.suffix for p in pathlib.Path.cwd().iterdir()])
# Counter({'.md': 2, '.txt': 4, '.pdf': 2, '.py': 1})

# iterdir()는 해당 디렉터리의 모든 파일을 이터레이터로 반환한다. Path 객체의 suffix는 .을 포함한 파일 확장자를 뜻한다.

# 참고 : 010 사용한 단어 개수를 구하려면? - collections.Counter

# 참고
# glob - 파일 검색: https://wikidocs.net/110619
# pathlib - 객체 지향 파일 시스템 경로: https://docs.python.org/ko/3/library/pathlib.html
# 하위 디렉터리 검색하기: https://wikidocs.net/39

In [24]:
import pathlib

for p in pathlib.Path.cwd().glob('*.txt'):
        new_p = p.parent.joinpath('archive', p.name)
        p.replace(new_p)

In [42]:
import pathlib, collections

print(collections.Counter([ p.suffix for p in pathlib.Path.cwd().iterdir() ]))
collections.Counter([ p.suffix for p in pathlib.Path.cwd().iterdir() ]).most_common()

Counter({'.ipynb': 7, '.txt': 4, '': 3})


[('.ipynb', 7), ('.txt', 4), ('', 3)]

## 036 디렉터리의 구성을 알려면? ― os.path

In [ ]:
# 036 디렉터리의 구성을 알려면? ― os.path
# -------------------------------------
# os.path는 경로명과 파일명에 대한 유용한 함수를 제공하는 모듈이다.

# 문제
# ----
# 특정 폴더(디렉터리)에 어떤 파이썬 파일이 있는지 알고자 한다. 
# 이럴 때 매개변수에 지정한 디렉터리의 파이썬 파일을 모두 출력하는 search(dirname) 함수는 어떻게 작성해야 할까? 
# 단, 하위 디렉터리의 파이썬 파일도 모두 출력해야 한다.


In [ ]:
# 풀이
# ----
# 디렉터리 경로를 입력으로 받아 파이썬 파일을 출력하는 search() 함수는 다음과 같다.

# [파일명: os_path_sample.py]

# import os

# def search(dirname):
#     filenames = os.listdir(dirname)
#     for filename rin filenames:
#         filepath = os.path.join(dirname, filename)
#         if os.path.isdir(filepath):
#             search(filepath)
#         elif os.path.isfile(filepath):
#             name, ext = os.path.splitext(filepath)
#             if ext == '.py': 
#                 print(filepath)

# search("c:/projects/pylib")  # c:/projects/pylib 디렉터리와 하위 디렉터리의 모든 .py 파일 출력
# 입력으로 받은 디렉터리의 모든 파일을 os.listdir()로 얻고 확장자가 .py이면 파일명을 출력한다. 
# os.listdir()로 얻은 경로가 파일이 아닌 디렉터리라면 다시 그 경로를 입력으로 하여 search() 함수를 재귀호출한다.

# 여기서 사용한 os 모듈과 os.path 모듈을 정리한 표는 다음과 같다.

# 사용 모듈	설명
# -------------
# os.listdir(path)	path 하위의 파일(디렉터리 포함)을 리스트로 반환한다.
# os.path.join(path, *paths)	path와 paths에 해당하는 모든 경로를 더하여 전체 경로를 반환한다. 
#                               이때 경로 구분자는 실행하는 운영체제에 따라 달라진다. (경로 구분자의 예: 유닉스는 /, 윈도우는 \\)
# os.path.isdir(path)	path의 디렉터리 여부를 반환한다.
# os.path.isfile(path)	path의 파일 여부를 반환한다.
# os.path.splitext(path)	path를 파일명과 확장자로 구분한다. (이때 확장자에는 마침표(.)를 포함한다.)


# 알아두면 좋아요
# pathlib를 사용하고 싶다면?
# -------------------------
# pathlib를 사용하여 만든 search() 함수는 다음과 같다.

# import pathlib

# def search(dirname):
#     for p in pathlib.Path(dirname).rglob('*.py'):
#         print(p)
# pathlib와 관련한 내용은 앞 절을 참고하자.

# 참고
# os.path - 일반적인 경로명 조작: https://docs.python.org/ko/3/library/os.path.html

In [49]:
import pathlib

def search(dirname):
    for p in pathlib.Path(dirname).rglob('*.ipynb'):
        print(p)
        
search('.')        

00. 들어가기 전에.ipynb
01. 텍스트 다루기.ipynb
02. 바이너리 데이터 다루기.ipynb
03. 다양한 데이터 다루기.ipynb
04. 수학과 숫자 다루기.ipynb
05. 함수형 프로그래밍 다루기.ipynb
06 . 파일과 디렉터리 다루기.ipynb
test_bed\03. 다양한 데이터 다루기.ipynb
test_bed\04. 수학과 숫자 다루기.ipynb
test_bed\05. 함수형 프로그래밍 다루기.ipynb
test_bed\06 . 파일과 디렉터리 다루기.ipynb


In [46]:
# import os

# def search(dirname):
#     filenames = os.listdir(dirname)
#     for filename rin filenames:
#         filepath = os.path.join(dirname, filename)
#         if os.path.isdir(filepath):
#             search(filepath)
#         elif os.path.isfile(filepath):
#             name, ext = os.path.splitext(filepath)
#             if ext == '.py': 
#                 print(filepath)

import os

def search(dirname):
    filenames = os.listdir(dirname)
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        if os.path.isdir(filepath):
            search(filepath)
        elif os.path.isfile(filepath):
            name, ext = os.path.splitext(filepath)
            if ext == '.ipynb':
                print(filepath)

search('.')            

.\00. 들어가기 전에.ipynb
.\01. 텍스트 다루기.ipynb
.\02. 바이너리 데이터 다루기.ipynb
.\03. 다양한 데이터 다루기.ipynb
.\04. 수학과 숫자 다루기.ipynb
.\05. 함수형 프로그래밍 다루기.ipynb
.\06 . 파일과 디렉터리 다루기.ipynb
.\test_bed\03. 다양한 데이터 다루기.ipynb
.\test_bed\04. 수학과 숫자 다루기.ipynb
.\test_bed\05. 함수형 프로그래밍 다루기.ipynb
.\test_bed\06 . 파일과 디렉터리 다루기.ipynb


## 037 여러 개의 파일을 한꺼번에 읽으려면? ― fileinput

In [ ]:
# 037 여러 개의 파일을 한꺼번에 읽으려면? ― fileinput
# -------------------------------------------------
# fileinput은 여러 개의 파일을 한꺼번에 처리할 때 사용하는 모듈이다.

# 문제
# ----
# 현재 디렉터리에 텍스트 파일(.txt)이 너무 많아 하나씩 열어 내용을 확인하려니 너무 번거롭다. 
# 이럴 때 모든 텍스트 파일을 읽어 그 내용을 한 줄씩 출력하려면 어떻게 코드를 작성해야 할까?


In [ ]:
# 풀이
# ----
# 여러 개의 파일을 한꺼번에 처리할 때는 다음처럼 fileinput 모듈을 사용하는 것이 가장 편리하다.

# [파일명: fileinput_sample.py]

# import fileinput
# import glob

# with fileinput.input(glob.glob("*.txt")) as f:
#     for line in f:
#         print(line)
# 파일 단위가 아니라 모든 txt 파일 전체를 하나의 파일처럼 처리하고자 fileinput.input(glob.glob("*.txt"))와 같이 사용했다.

# 참고
# fileinput - 여러 입력 스트림에서 줄을 이터레이트 하기: https://docs.python.org/ko/3/library/fileinput.html
# 파일 읽고 쓰기: https://wikidocs.net/26

In [67]:
import fileinput
import glob

def multi_file_read(*args):
    
    if args: dirname = args[0]
    else: dirname = os.getcwd()

    with fileinput.input(glob.glob(f'{dirname}/*.txt')) as f:
        for line in f:
            print(line)

multi_file_read('./archive')
multi_file_read()

1
2
3
4
1
2
3
4


## 038 디렉터리와 파일을 비교하려면? ― filecmp

In [ ]:
# 038 디렉터리와 파일을 비교하려면? ― filecmp
# -----------------------------------------
# filecmp는 파일 두 개 또는 디렉터리 두 곳을 비교할 때 사용하는 모듈이다.

# 문제
# ----
# 두 디렉터리의 차이를 다음과 같은 조건으로 확인하는 프로그램은 어떻게 작성해야 할까?

# a 디렉터리에만 있고 b 디렉터리에는 없는 파일은 다음과 같이 출력한다.
# a: 파일명

# b 디렉터리에만 있고 a 디렉터리에는 없는 파일은 다음과 같이 출력한다.
# b: 파일명

# a 디렉터리와 b 디렉터리에 모두 있으나 파일의 내용이 서로 다를 때는 다음과 같이 출력한다.
# x: 파일명

In [ ]:
# 풀이
# ----
# filecmp.dircmp()를 사용하면 문제에서 요구하는 프로그램을 쉽게 만들 수 있다.

# [파일명: filecmp_sample.py]

# import filecmp

# fd = filecmp.dircmp('a', 'b')

# for a in fd.left_only:
#     print("a: %s" % a)

# for b in fd.right_only:
#     print("b: %s" % b)

# for x in fd.diff_files:
#     print("x: %s" % x)

# 먼저 fd = filecmp.dircmp('a', 'b')처럼 a, b 디렉터리를 비교하고자 fd 객체를 생성한다. 
# 그리고 a 디렉터리에만 있는 파일은 fd.left_only, b 디렉터리에만 있는 파일은 fd.right_only, 
# 두 곳 모두에 있지만 내용이 다른 파일은 fd.diff_files로 구할 수 있다.

# 참고
# filecmp- 파일과 디렉터리 비교: https://docs.python.org/ko/3/library/filecmp.html

In [68]:
import filecmp

fd = filecmp.dircmp('a', 'b')

for a in fd.left_only:
    print('a: %s' %a)

## 039 임시로 만든 파일을 이용하려면? ― tempfile

In [ ]:
# 039 임시로 만든 파일을 이용하려면? ― tempfile
# ---------------------------------------------
# tempfile은 임시 파일을 만들 때 사용하는 모듈이다.

# 문제
# ---
# 다음은 파일 객체를 입력으로 받아 해당 파일 각 줄의 숫자를 모두 더하고 나서 그 수를 반환하는 sumfile() 함수이다.

# def sumfile(f):
#     result = 0
#     for line in f.readlines():
#         num = int(line)
#         result += num
#     return result

# 1~100 사이의 숫자 10개를 무작위로 생성하여 줄 단위로 임시 파일에 기록한 후 sumfile() 
# 함수를 이용하여 그 숫자의 합을 출력하는 프로그램은 어떻게 작성해야 할까? 
# 단, 생성한 파일은 함수 실행 후 삭제해야 한다.


In [19]:
import random

with open('/temp/test.txt', 'wt') as f:
    for data in random.sample(range(1, 101), 10):
        f.write(f'{data}\n')

In [5]:
def sumfile(f):
    result = 0
    for line in f.readlines():
        num = int(line)
        result += num
    return result

with open('/temp/test.txt', 'rt') as f:
    result = sumfile(f)

print(result)

575


In [ ]:
# 풀이
# ----
# 무작위로 숫자를 생성할 때는 random.randint() 함수를 사용하고 파일을 임시로 생성할 때는 tempfile을 사용한다.

# 참고: 021 로또 번호를 뽑으려면? - random

# [파일명: tempfile_sample.py]

# import random
# import tempfile


# def sumfile(f):
#     result = 0
#     for line in f.readlines():
#         num = int(line)
#         result += num
#     return result


# tf = tempfile.TemporaryFile(mode='w+')
# for i in range(10):
#     num = random.randint(1, 100)
#     tf.write(str(num))
#     tf.write("\n")

# tf.seek(0)  # 파일 오프셋을 처음으로 이동
# result = sumfile(tf)
# tf.close()

# print(result)

# tf = tempfile.TemporaryFile(mode='w+')처럼 tempfile 모듈을 사용하면 임시 파일을 쉽게 만들 수 있다. 
# mode='w+'는 쓰기와 읽기를 위해 파일을 생성한다는 의미이다. 
# 여기서는 무작위로 생성한 10개의 숫자를 파일에 써야 하고, 또 sumfile() 함수에서는 파일을 읽어야 하므로 mode='w+'를 사용하였다. 
# 파일 쓰기가 끝나면 파일의 오프셋(offset)이 파일의 끝을 가리키므로 tf.seek(0)를 수행하여 처음부터 파일을 읽을 수 있도록 했다. 
# 임시 파일은 tf.close()가 수행되거나 파이썬 프로세스가 종료되면 자동으로 삭제된다.

# 알아두면 좋아요
# NamedTemporaryFile()
# --------------------
# TemporaryFile() 대신 NamedTemporaryFile()을 사용하면 생성되는 임시 파일에 파일 시스템에서 볼 수 있는 이름이 반드시 붙게 된다. 
# 예를 들어 유닉스에서는 앞 코드의 tf.close()가 실행되기 전에 input()과 같은 사용자 입력 대기 명령줄을 추가하면 사용자 입력이 되어 
# 프로그램이 종료되기 전에 /tmp 디렉터리에 생성된 임시 파일을 눈으로 확인할 수 있다. 
# NamedTemporaryFile()은 이 점만 제외하면 TemporaryFile()과 기능이 똑같다.

# 참고
# tempfile - 임시 파일과 디렉터리 생성: https://docs.python.org/ko/3/library/tempfile.html

In [34]:
import random
import tempfile

def sumfile(f):
    result = 0
    for line in f.readlines():
        num = int(line)
        result += num
    return result


rand_num = random.sample(range(1,101), 6)
with tempfile.TemporaryFile(mode='w+') as tf:
    for data in rand_num:
        tf.write(f'{data}\n')
    tf.seek(0)
    result = sumfile(tf)
    print(result)

348


In [35]:
import random
import tempfile
import time

def sumfile(f):
    result = 0
    for line in f.readlines():
        num = int(line)
        result += num
    return result


rand_num = random.sample(range(1,101), 6)
with tempfile.NamedTemporaryFile(mode='w+') as tf:
    for data in rand_num:
        tf.write(f'{data}\n')
    tf.seek(0)
    result = sumfile(tf)
print(result)

253


## 040 파일을 찾으려면? ― glob

In [ ]:
# 040 파일을 찾으려면? ― glob
# ---------------------------
# glob는 패턴(유닉스 셸이 사용하는 규칙)을 이용하여 파일을 검색할 때 사용하는 모듈이다.

# 문제
# ----
# 특정 디렉터리에 어떤 텍스트 파일이 있는지 확인하고자 한다. 
# 이를 위해 현재 디렉터리와 하위 디렉터리의 모든 텍스트 파일(*.txt)을 찾아 해당 파일명을 출력하려면 어떻게 프로그램을 작성해야 할까?

In [51]:
import glob

for filename in glob.glob('./**/*.ipynb', recursive=True):
    print(filename)

.\00. 들어가기 전에.ipynb
.\01. 텍스트 다루기.ipynb
.\02. 바이너리 데이터 다루기.ipynb
.\03. 다양한 데이터 다루기.ipynb
.\04. 수학과 숫자 다루기.ipynb
.\05. 함수형 프로그래밍 다루기.ipynb
.\06 . 파일과 디렉터리 다루기.ipynb
.\test_bed\03. 다양한 데이터 다루기.ipynb
.\test_bed\04. 수학과 숫자 다루기.ipynb
.\test_bed\05. 함수형 프로그래밍 다루기.ipynb
.\test_bed\06 . 파일과 디렉터리 다루기.ipynb


In [ ]:
# 풀이
# ----
# 다음은 glob 모듈을 사용한 문제 풀이이다.

# [파일명: glob_sample.py]

# import glob

# for filename in glob.glob("**/*.txt", recursive=True):
#     print(filename)
# 여기서 사용한 **/*.txt 패턴과 recursive=True는 하위 경로를 포함한 모든 텍스트 파일을 검색한다는 뜻이다. 
# 만약 현재 디렉터리의 텍스트 파일만 검색하려면 *.txt 패턴을 사용하면 된다.

# **/*.txt와 같은 재귀 패턴은 파이썬 3.5 버전부터 사용할 수 있다.

In [48]:
import glob

for filename in glob.glob("**/*.txt", recursive=True):
    print(filename)

test1.txt
test2.txt
test3.txt
test4.txt
archive\test1.txt
archive\test2.txt
archive\test3.txt
archive\test4.txt


## 041 특정 파일만 찾으려면? ― fnmatch

In [ ]:
# 041 특정 파일만 찾으려면? ― fnmatch
# ----------------------------------
# fnmatch는 파일 중에서 특정 패턴(유닉스 셸이 사용하는 규칙)과 일치하는 파일을 찾을 때 사용하는 모듈이다.

# 문제
# ----
# 현재 디렉터리의 모든 파일 중에서 다음과 같은 규칙을 따르는 파일을 모두 찾아 출력하고자 한다. 어떻게 프로그램을 만들어야 할까?

# - 파일명은 a로 시작한다.
# - 확장자는 파이썬 파일을 의미하는 .py 이다.
# - 확장자를 제외한 파일명의 길이는 5이다.
# - 파일명의 마지막 5번째 문자는 숫자이다.

In [ ]:
# 풀이
# ----
# 다음은 fnmatch를 사용한 문제 풀이이다.

# [파일명: fnmatch_sample.py]

# import fnmatch
# import os

# for filename in os.listdir('.'):
#     if fnmatch.fnmatch(filename, 'a???[0-9].py'):
#         print(filename)

# fnmatch.fnmatch(filename, pattern)은 filename 문자열이 pattern 문자열과 일치하는지를 검사하여 True나 False를 반환한다.

# fnmatch에 사용할 수 있는 패턴에는 다음과 같은 것이 있다.

# 패턴	의미
# *	모든 것과 일치
# ?	모든 단일 문자와 일치
# [seq]	seq의 모든 문자와 일치
# [!seq]	seq에 없는 모든 문자와 일치
# 따라서 fnmatch에 사용한 a???[0-9].py 패턴은 문제의 요구 조건과 정확히 일치한다.

# 참고
# fnmatch - 유닉스 파일명 패턴 일치: https://docs.python.org/ko/3/library/fnmatch.html

In [5]:
import fnmatch
import os

for filename in os.listdir('.'):
    if fnmatch.fnmatch(filename, 'a???[1-9].txt'):
        print(filename)

aest4.txt


## 042 파일에서 무작위로 한 줄만 가져오려면? ― linecache

In [ ]:
# 042 파일에서 무작위로 한 줄만 가져오려면? ― linecache
# --------------------------------------------------
# linecache는 파일에서 원하는 줄의 값을 읽을 때 사용하는 모듈이다. 
# 이때 캐시를 사용하여 파일에서 여러 줄을 읽는 일반적인 상황을 내부적으로 최적화한다.

# 문제
# 다음은 속담 퀴즈 프로그램에 사용할 속담 100개를 저장한 saying.txt 파일이다.

# 파일 내려받기: https://github.com/pahkey/pylib/blob/main/ch06/saying.txt

# [파일명: saying.txt]

# 가는 날이 장날이다
# 가는 말이 고와야 오는 말이 곱다
# 가랑비에 옷 젖는 줄 모른다
# 가랑잎이 솔잎더러 바스락거린다고 한다
# 가재는 게 편이라
# 가지 많은 나무에 바람 잘 날 없다
# 간에 가 붙고 쓸개에 가 붙는다
# 간에 기별도 안 간다
# 간이 콩알만해지다
# 갈수록 태산
# 값싼 것이 비지떡
# 같은 값이면 다홍치마
# 개구리 올챙이 적 생각을 못한다
# 개밥에 도토리
# 개천에서 용 난다
# 고기는 씹어야 맛이요, 말은 해야 맛이라
# 고래 싸움에 새우 등 터진다
# 고양이 목에 방울 달기
# 공든 탑이 무너지랴
# 구더기 무서워 장 못 담글까
# 구슬이 서 말이라도 꿰어야 보배라
# 귀에 걸면 귀걸이, 코에 걸면 코걸이
# 그림의 떡
# 금강산도 식후경
# 뛰는 놈 위에 나는 놈 있다
# 까마귀 날자 배 떨어진다
# 꿩 대신 닭
# 꿩 먹고 알 먹기
# 남의 잔치에 감 놓아라 배 놓아라 한다
# 낫 놓고 기역자도 모른다
# 낮말은 새가 듣고 밤말은 쥐가 듣는다
# 내 코가 석 자
# 누워서 침 뱉기
# 늦게 배운 도둑이 날 새는 줄 모른다
# 다 된 죽에 코 풀기
# 달면 삼키고 쓰면 뱉는다
# 닭 잡아 먹고 오리발 내민다
# 도둑이 제 발 저리다
# 돌다리도 두들겨 보고 건너라
# 되로 주고 말로 받는다
# 등잔 밑이 어둡다
# 땅 짚고 헤엄치기
# 똥 묻은 개가 겨 묻은 개 나무란다
# 마른 하늘에 날벼락
# 말 한마디에 천 냥 빚도 갚는다
# 목구멍이 포도청
# 못된 송아지 엉덩이에 뿔 난다
# 믿는 도끼에 발등 찍힌다
# 밑 빠진 독에 물 붓기
# 바늘 도둑이 소 도둑 된다
# 배보다 배꼽이 더 크다
# 백지장도 맞들면 낫다
# 벼룩의 간 빼먹기
# 병 주고 약 준다
# 보기 좋은 떡이 먹기도 좋다
# 빛 좋은 개살구
# 사공이 많으면 배가 산으로 올라간다
# 새발의 피
# 서당 개 삼 년에 풍월을 읊는다
# 세 살 버릇 여든까지 간다
# 소문난 잔치에 먹을 것 없다
# 소 잃고 외양간 고친다
# 쇠뿔도 단김에 빼랬다
# 수박 겉 핥기
# 식은 죽 먹기
# 십 년이면 강산도 변한다
# 아는 길도 물어 가라
# 아니 땐 굴뚝에 연기 나랴
# 아닌 밤중에 홍두깨
# 약방에 감초
# 어물전 망신은 꼴뚜기가 시킨다
# 열 길 물 속은 알아도 한 길 사람 속은 모른다
# 열 번 찍어 아니 넘어가는 나무 없다
# 오뉴월 감기는 개도 아니 앓는다
# 오르지 못할 나무는 쳐다보지도 말아라
# 옥의 티
# 우물에 가서 숭늉 찾는다
# 울며 겨자 먹기
# 원수는 외나무 다리에서 만난다
# 원숭이도 나무에서 떨어진다
# 윗물이 맑아야 아랫물도 맑다
# 자라 보고 놀란 가슴 솥뚜껑 보고 놀란다
# 자랄 나무는 떡잎부터 알아본다
# 작은 고추가 더 맵다
# 종로에서 뺨 맞고 한강 가서 눈 흘긴다
# 좋은 약은 입에 쓰다
# 쥐구멍에도 볕 들 날이 있다
# 지렁이도 밟으면 꿈틀한다
# 천 리 길도 한 걸음부터
# 칼로 물 베기
# 콩 심은 데 콩 나고 팥 심은 데 팥 난다
# 티끌 모아 태산
# 핑계 없는 무덤 없다
# 하늘의 별 따기
# 하늘이 무너져도 솟아날 구멍이 있다
# 하룻강아지 범 무서운 줄 모른다
# 한 귀로 듣고 한 귀로 흘린다
# 한 술 밥에 배 부르랴
# 함흥차사라
# 호랑이도 제 말 하면 온다
# 이 파일의 100개 속담 중 한 개를 무작위로 선택해서 출력하는 프로그램은 어떻게 작성해야 할까?

In [65]:
saying_text = """
가는 날이 장날이다
가는 말이 고와야 오는 말이 곱다
가랑비에 옷 젖는 줄 모른다
가랑잎이 솔잎더러 바스락거린다고 한다
가재는 게 편이라
가지 많은 나무에 바람 잘 날 없다
간에 가 붙고 쓸개에 가 붙는다
간에 기별도 안 간다
간이 콩알만해지다
갈수록 태산
값싼 것이 비지떡
같은 값이면 다홍치마
개구리 올챙이 적 생각을 못한다
개밥에 도토리
개천에서 용 난다
고기는 씹어야 맛이요, 말은 해야 맛이라
고래 싸움에 새우 등 터진다
고양이 목에 방울 달기
공든 탑이 무너지랴
구더기 무서워 장 못 담글까
구슬이 서 말이라도 꿰어야 보배라
귀에 걸면 귀걸이, 코에 걸면 코걸이
그림의 떡
금강산도 식후경
뛰는 놈 위에 나는 놈 있다
까마귀 날자 배 떨어진다
꿩 대신 닭
꿩 먹고 알 먹기
남의 잔치에 감 놓아라 배 놓아라 한다
낫 놓고 기역자도 모른다
낮말은 새가 듣고 밤말은 쥐가 듣는다
내 코가 석 자
누워서 침 뱉기
늦게 배운 도둑이 날 새는 줄 모른다
다 된 죽에 코 풀기
달면 삼키고 쓰면 뱉는다
닭 잡아 먹고 오리발 내민다
도둑이 제 발 저리다
돌다리도 두들겨 보고 건너라
되로 주고 말로 받는다
등잔 밑이 어둡다
땅 짚고 헤엄치기
똥 묻은 개가 겨 묻은 개 나무란다
마른 하늘에 날벼락
말 한마디에 천 냥 빚도 갚는다
목구멍이 포도청
못된 송아지 엉덩이에 뿔 난다
믿는 도끼에 발등 찍힌다
밑 빠진 독에 물 붓기
바늘 도둑이 소 도둑 된다
배보다 배꼽이 더 크다
백지장도 맞들면 낫다
벼룩의 간 빼먹기
병 주고 약 준다
보기 좋은 떡이 먹기도 좋다
빛 좋은 개살구
사공이 많으면 배가 산으로 올라간다
새발의 피
서당 개 삼 년에 풍월을 읊는다
세 살 버릇 여든까지 간다
소문난 잔치에 먹을 것 없다
소 잃고 외양간 고친다
쇠뿔도 단김에 빼랬다
수박 겉 핥기
식은 죽 먹기
십 년이면 강산도 변한다
아는 길도 물어 가라
아니 땐 굴뚝에 연기 나랴
아닌 밤중에 홍두깨
약방에 감초
어물전 망신은 꼴뚜기가 시킨다
열 길 물 속은 알아도 한 길 사람 속은 모른다
열 번 찍어 아니 넘어가는 나무 없다
오뉴월 감기는 개도 아니 앓는다
오르지 못할 나무는 쳐다보지도 말아라
옥의 티
우물에 가서 숭늉 찾는다
울며 겨자 먹기
원수는 외나무 다리에서 만난다
원숭이도 나무에서 떨어진다
윗물이 맑아야 아랫물도 맑다
자라 보고 놀란 가슴 솥뚜껑 보고 놀란다
자랄 나무는 떡잎부터 알아본다
작은 고추가 더 맵다
종로에서 뺨 맞고 한강 가서 눈 흘긴다
좋은 약은 입에 쓰다
쥐구멍에도 볕 들 날이 있다
지렁이도 밟으면 꿈틀한다
천 리 길도 한 걸음부터
칼로 물 베기
콩 심은 데 콩 나고 팥 심은 데 팥 난다
티끌 모아 태산
핑계 없는 무덤 없다
하늘의 별 따기
하늘이 무너져도 솟아날 구멍이 있다
하룻강아지 범 무서운 줄 모른다
한 귀로 듣고 한 귀로 흘린다
한 술 밥에 배 부르랴
함흥차사라
호랑이도 제 말 하면 온다
"""

import random 

with open('./saying.txt', 'wt') as f:
    for saying in saying_text.split('\n'):
        if saying:
            f.write(f'{saying}\n')


with open('./saying.txt', 'rt') as f:
    lines = f.readlines()
    
search_line = random.randint(0,99)
print(lines[search_line])

가는 말이 고와야 오는 말이 곱다



In [ ]:
# 풀이
# 다음은 linecache를 사용한 문제 풀이이다.

# [파일명: linecache_sample.py]

# import linecache
# import random

# no = random.randint(1, 100)
# print(linecache.getline('saying.txt', no))

# linecache.getline(filename, lineno)는 지정한 줄을 파일에서 읽어 반환한다. 
# 이때 메모리 캐시를 사용하므로 linecache.getline()을 여러 번 호출하더라도 캐시에 저장한 값을 반환하므로 속도가 빠르다.

# 참고
# linecache - 텍스트 줄에 대한 무작위 액세스: https://docs.python.org/ko/3/library/linecache.html
# random 더 알아보기: https://wikidocs.net/33#random

In [109]:
import linecache
import random

no = random.randint(1, 100)
print(linecache.getline('saying.txt', no))

콩 심은 데 콩 나고 팥 심은 데 팥 난다



## 043 파일을 복사하거나 이동하려면? ― shutil

In [ ]:
# 043 파일을 복사하거나 이동하려면? ― shutil
# ----------------------------------------
# shutil은 파일을 복사(copy)하거나 이동(move)할 때 사용하는 모듈이다.

# 문제
# ----
# 작업 중인 파일을 자동으로 백업하는 기능을 구현하고자 c:\doit\a.txt 파일을 c:\temp\a.txt.bak이라는 
# 이름으로 복사하는 프로그램을 만들고자 한다. 어떻게 만들어야 할까? 
# c:\doit 디렉터리에 a.txt 파일을 만드는 중이며 백업용 c:\temp 디렉터리는 이미 만들었다고 가정한다.

In [ ]:
# 풀이
# ----
# 다음은 shutil을 사용한 문제 풀이이다.

# [파일명: shutil_sample.py]

# import shutil

# shutil.copy("c:/doit/a.txt", "c:/temp/a.txt.bak")
# 알아두면 좋아요

# shutil.move
# 휴지통으로 삭제하는 기능을 구현하고자 c:\doit\a.txt 파일을 c:\temp\a.txt로 이동하려면 다음과 같이 코드를 작성한다.

# import shutil

# shutil.move("c:/doit/a.txt", "c:/temp/a.txt")

# 참고
# shutil - 고수준 파일 연산: https://docs.python.org/ko/3/library/shutil.html

In [113]:
import shutil

# shutil.copy('./test3.txt','c:/temp/test.txt.bak')
shutil.move('c:/temp/test.txt','./test3.txt')

'./test3.txt'